***GENERATED CODE FOR join PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***OPERATION FUNCTIONS***

In [ ]:
from pyspark.sql import functions as F


def QueryBuilder(spark, joindatalist, paths):
    listOfDfnames = {}
    for path in paths:
        listOfDfnames[path['sourcename']] = spark.read.options(
            header='true', inferschema='true', delimiter=',').csv('dbfs:' + path['sourcepath'])
    for join in joindatalist:
        dfA = listOfDfnames[join['sourceA']['sourcename']]
        dfB = listOfDfnames[join['sourceB']['sourcename']]
        f1 = 'a.' + join['sourceA']['fieldname'].replace(" ", "")
        f2 = 'b.' + join['sourceB']['fieldname'].replace(" ", "")
        listOfDfnames[join['sourceA']['sourcename'] + '_' + join['sourceB']['sourcename']] = removeIfDuplicates(dfA.alias('a').join(
            dfB.alias('b'), F.col(f1) == F.col(f2),
            how=join['joinName']))
        joinedDf = listOfDfnames[join['sourceA']
                                 ['sourcename'] + '_' + join['sourceB']['sourcename']]
    df = removeIfDuplicates(joinedDf)
    print(display(df.limit(2).toPandas()))
    return df


def removeIfDuplicates(joinedDf):
    cols_new = []
    seen = set()
    for c in joinedDf.columns:
        cols_new.append('{}_dup'.format(c) if c in seen else c)
        seen.add(c)
    df = joinedDf.toDF(*cols_new).select(*
                                         [c for c in cols_new if not c.endswith('_dup')])
    return df


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run joinHooks.ipynb
try:
	#sourcePreExecutionHook()
		fetalhealth = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/fetal_health.csv', 'filename': 'fetal_health.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")



	fetalhealth = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/fetal_health2.csv', 'filename': 'fetal_health2.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'results_url': 'http://13.59.224.229:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run joinHooks.ipynb
try:
	#operationPreExecutionHook()

	join = QueryBuilder(spark,[{"sourceA": {"sourceid": "60f12fa18238f17400db4dae", "fieldname": "accelerations", "sourceinfo": {"source": "hdfs", "sourceAttributes": {"url": "/FileStore/platform/uploadedSourceFiles/fetal_health.csv"}}, "sourcename": "fetal_health"}, "sourceB": {"sourceid": "60f12fa68238f17400db4daf", "fieldname": "accelerations", "sourceinfo": {"source": "hdfs", "sourceAttributes": {"url": "/FileStore/platform/uploadedSourceFiles/fetal_health2.csv"}}, "sourcename": "fetal_health2"}, "joinName": "inner"}],[{'sourcename': 'fetal_health', 'sourceinfo': {'source': 'hdfs', 'sourceAttributes': {'url': '/FileStore/platform/uploadedSourceFiles/fetal_health.csv'}}}, {'sourcename': 'fetal_health2', 'sourceinfo': {'source': 'hdfs', 'sourceAttributes': {'url': '/FileStore/platform/uploadedSourceFiles/fetal_health2.csv'}}}])
	#operationPostExecutionHook(join)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run joinHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(join,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "baseline value", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "18394", "mean": "139.75", "stddev": "13.47", "min": "114", "max": "159", "missing": "0"}, "updatedLabel": "baseline value"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "accelerations", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18394", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "accelerations"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "fetal_movement", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18394", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "fetal_movement"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "uterine_contractions", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18394", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "uterine_contractions"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "light_decelerations", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18394", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "light_decelerations"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "severe_decelerations", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "18394", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "severe_decelerations"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "prolongued_decelerations", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18394", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "prolongued_decelerations"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "abnormal_short_term_variability", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "18394", "mean": "48.94", "stddev": "19.83", "min": "16", "max": "87", "missing": "0"}, "updatedLabel": "abnormal_short_term_varia..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "mean_value_of_short_term_variability", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18394", "mean": "1.14", "stddev": "1.62", "min": "0.0", "max": "7.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "mean_value_of_short_term_..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "percentage_of_time_with_abnormal_long_term_variability", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "18394", "mean": "17.39", "stddev": "23.48", "min": "0", "max": "79", "missing": "0"}, "updatedLabel": "percentage_of_time_with_a..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "mean_value_of_long_term_variability", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "18394", "mean": "9.91", "stddev": "6.31", "min": "0.0", "max": "35.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "mean_value_of_long_term_v..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_width", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "18394", "mean": "80.54", "stddev": "50.22", "min": "10", "max": "150", "missing": "0"}, "updatedLabel": "histogram_width"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_min", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "18394", "mean": "89.98", "stddev": "37.6", "min": "50", "max": "159", "missing": "0"}, "updatedLabel": "histogram_min"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_max", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "18394", "mean": "170.52", "stddev": "20.98", "min": "126", "max": "200", "missing": "0"}, "updatedLabel": "histogram_max"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_number_of_peaks", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "18394", "mean": "4.49", "stddev": "3.93", "min": "0", "max": "14", "missing": "0"}, "updatedLabel": "histogram_number_of_peaks"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_number_of_zeroes", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "18394", "mean": "0.36", "stddev": "0.84", "min": "0", "max": "10", "missing": "0"}, "updatedLabel": "histogram_number_of_zeroe..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_mode", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "18394", "mean": "141.32", "stddev": "18.23", "min": "71", "max": "170", "missing": "0"}, "updatedLabel": "histogram_mode"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_mean", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "18394", "mean": "139.09", "stddev": "18.31", "min": "99", "max": "172", "missing": "0"}, "updatedLabel": "histogram_mean"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_median", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "18394", "mean": "141.73", "stddev": "17.41", "min": "102", "max": "174", "missing": "0"}, "updatedLabel": "histogram_median"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_variance", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "18394", "mean": "19.55", "stddev": "34.99", "min": "0", "max": "215", "missing": "0"}, "updatedLabel": "histogram_variance"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_tendency", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "18394", "mean": "0.51", "stddev": "0.55", "min": "-1", "max": "1", "missing": "0"}, "updatedLabel": "histogram_tendency"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "fetal_health", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "18394", "mean": "1.63", "stddev": "0.76", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "fetal_health"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run joinHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["baseline value", "accelerations", "fetal_movement", "uterine_contractions", "light_decelerations", "severe_decelerations", "prolongued_decelerations", "abnormal_short_term_variability", "mean_value_of_short_term_variability", "percentage_of_time_with_abnormal_long_term_variability", "mean_value_of_long_term_variability", "histogram_width", "histogram_min", "histogram_max", "histogram_number_of_peaks", "histogram_number_of_zeroes", "histogram_mode", "histogram_mean", "histogram_median", "histogram_variance", "histogram_tendency"], "fetal_health")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

